In [22]:
import dynamicrouting_summary as dr
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import oursin as urchin

In [2]:
dfs = dr.get_dfs()

In [3]:
dfs.keys()

KeysView(LazyDict(keys=['session', 'subject', 'epochs', 'trials', 'performance', 'vis_rf_mapping', 'aud_rf_mapping', 'optotagging', 'invalid_times', 'devices', 'electrode_groups', 'electrodes']))

In [4]:
electrodes_original = dfs['electrodes']
electrodes_original

,location,group_name,structure,x,y,z,channel,rel_x,rel_y,reference,imp,session_idx,date,subject_id,session_id,is_ephys,is_templeton,is_training,is_dynamic_routing,is_opto
0,int,probeA,int,5975.0,4725.0,4050.0,0,27,0,tip,150000.0,0,2023-08-09,660023,660023_2023-08-09_0,True,False,False,True,False
1,int,probeA,int,5975.0,4725.0,4050.0,1,59,0,tip,150000.0,0,2023-08-09,660023,660023_2023-08-09_0,True,False,False,True,False
2,int,probeA,int,5975.0,4675.0,4075.0,2,11,20,tip,150000.0,0,2023-08-09,660023,660023_2023-08-09_0,True,False,False,True,False
3,int,probeA,int,5975.0,4675.0,4075.0,3,43,20,tip,150000.0,0,2023-08-09,660023,660023_2023-08-09_0,True,False,False,True,False
4,int,probeA,int,5975.0,4650.0,4075.0,4,27,40,tip,150000.0,0,2023-08-09,660023,660023_2023-08-09_0,True,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100987,unannotated,probeF,None,NaN,NaN,NaN,379,43,3780,tip,150000.0,0,2023-11-30,690706,690706_2023-11-30_0,True,False,False,True,False
100988,unannotated,probeF,None,NaN,NaN,NaN,380,27,3800,tip,150000.0,0,2023-11-30,690706,690706_2023-11-30_0,True,False,False,True,False
100989,unannotated,probeF,None,NaN,NaN,NaN,381,59,3800,tip,150000.0,0,2023-11-30,690706,690706_2023-11-30_0,True,False,False,True,False
100990,unannotated,probeF,None,NaN,NaN,NaN,382,11,3820,tip,150000.0,0,2023-11-30,690706,690706_2023-11-30_0,True,False,False,True,False


In [5]:
import pyarrow.parquet as pq
import npc_lims

units = pq.ParquetDataset(npc_lims.get_cache_path('units'))
units.schema

num_spikes: double
firing_rate: double
presence_ratio: double
snr: double
isi_violations_ratio: double
isi_violations_count: double
rp_contamination: double
rp_violations: double
sliding_rp_violation: double
amplitude_cutoff: double
drift_ptp: double
drift_std: double
drift_mad: double
isolation_distance: double
l_ratio: double
d_prime: double
peak_to_valley: double
peak_trough_ratio: double
half_width: double
repolarization_slope: double
recovery_slope: double
electrode_group_name: string
peak_channel: int64
cluster_id: int64
default_qc: bool
amplitude: double
channels: list<element: int64>
  child 0, element: int64
unit_id: string
group_name: string
ccf_ap: double
ccf_dv: double
ccf_ml: double
structure: string
location: string
peak_electrode: int64
spike_times: list<element: double>
  child 0, element: double
obs_intervals: list<element: list<element: double>>
  child 0, element: list<element: double>
      child 0, element: double
electrodes: list<element: int64>
  child 0, element

In [6]:
units_structures = units.read(columns=['structure', 'location', 'peak_channel', 'subject_id', 'session_idx', 'date']).to_pandas()
units_structures_with_bools = dr.add_bool_columns(units_structures)
units_structures_with_bools.dropna(inplace=True)
units_structures_with_bools

,structure,location,peak_channel,subject_id,session_idx,date,session_id,is_ephys,is_templeton,is_training,is_dynamic_routing,is_opto
0,CA1,CA1,162,660023,0,2023-08-09,660023_2023-08-09_0,True,False,False,True,False
1,CA1,CA1,138,660023,0,2023-08-09,660023_2023-08-09_0,True,False,False,True,False
2,CA1,CA1,157,660023,0,2023-08-09,660023_2023-08-09_0,True,False,False,True,False
3,CA1,CA1,154,660023,0,2023-08-09,660023_2023-08-09_0,True,False,False,True,False
4,CA1,CA1,154,660023,0,2023-08-09,660023_2023-08-09_0,True,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...
57777,VPM,VPM,0,681532,0,2023-10-18,681532_2023-10-18_0,True,False,False,True,False
57778,VPM,VPM,0,681532,0,2023-10-18,681532_2023-10-18_0,True,False,False,True,False
57779,VPM,VPM,0,681532,0,2023-10-18,681532_2023-10-18_0,True,False,False,True,False
57780,cing,cing,244,681532,0,2023-10-18,681532_2023-10-18_0,True,False,False,True,False


In [7]:
import ipywidgets

electrodes = electrodes_original.dropna()
#electrodes = electrodes[~(electrodes['session_id'].str.contains('662892'))]
electrodes = electrodes[~(electrodes['structure'].str.islower())]
electrodes = electrodes[electrodes['is_dynamic_routing'] == True]

In [8]:
unique_areas = electrodes['structure'].unique()
unique_areas.sort()
area_dropdown = ipywidgets.Dropdown(options=unique_areas, value=unique_areas[0], description='CCF Structure Area', disabled=False)
area_dropdown

Dropdown(description='CCF Structure Area', options=('ACAd', 'ACAv', 'AD', 'AId', 'AIv', 'AON', 'APN', 'AQ', 'A…

In [15]:
area = area_dropdown.value
units_structures_with_bools_dr = units_structures_with_bools[units_structures_with_bools['is_dynamic_routing'] == True]

In [21]:
units_structures_with_bools_dr_area = units_structures_with_bools_dr[units_structures_with_bools_dr['structure'] == area]
electrodes_structures_area = electrodes[electrodes['structure'] == area]

print(f"{area}: {len(electrodes_structures_area['subject_id'].unique())} mice")
print(f"{area}: {len(electrodes_structures_area['session_id'].unique())} sessions")
print(f"{area}: {len(units_structures_with_bools_dr_area)} units")

ACAd: 4 mice
ACAd: 9 sessions
ACAd: 1217 units


In [31]:
urchin.setup()

(urchin) Client is already connected. Use ID: acfc43ef


In [32]:
urchin.ccf25.load()

(Warning) Atlas was already loaded, the renderer can have issues if you try to load an atlas twice.


In [25]:
urchin.ccf25.root.set_visibility(True)
urchin.ccf25.root.set_material('transparent-lit')
urchin.ccf25.root.set_alpha(0.15)
urchin.ccf25.root.set_color("#000000")

In [28]:
neurons = urchin.particles.create(len(electrodes_structures_area))
positions_list = electrodes_structures_area[['x', 'z', 'y']].values.tolist()

urchin.particles.set_positions(neurons, positions_list)

In [29]:
area_list = urchin.ccf25.get_areas([area_dropdown.value])

urchin.ccf25.set_visibilities(area_list, True, urchin.utils.Side.LEFT)
urchin.ccf25.set_materials(area_list, 'transparent-unlit', "left")
urchin.ccf25.set_alphas(area_list, 0.2, "left")